In [1]:
import pandas as pd
import requests
import os
import datetime
import json
import time
import ccxt
import numpy as np
import threading
from algos.daddy.huobi.HuobiDMService import HuobiDM


In [9]:
apiKey = os.getenv('BITMEX_ID')
apiSecret = os.getenv('BITMEX_SECRET')

exchange = ccxt.bitmex({
                    'apiKey': apiKey,
                    'secret': apiSecret,
                    'enableRateLimit': True,
                })
limit = 500
since = exchange.parse8601("2020-12-15 00:00:00")

In [10]:
since

1607990400000

In [11]:
all_trades = []

while True:
    curr_trades = exchange.fetchMyTrades(symbol='BTC/USD', limit=limit, since=since)
    since = exchange.parse8601(curr_trades[-1]['datetime'])
    all_trades = all_trades + curr_trades

    if len(curr_trades) < limit:
        break


trades = pd.DataFrame([trade['info'] for trade in all_trades])
trades['transactTime'] = pd.to_datetime(trades['transactTime'])
trades['transactTime'] = trades['transactTime'].dt.tz_localize(None)
trades = trades[['transactTime', 'symbol', 'side', 'price','lastQty', 'execComm', 'execType', 'text']]
funding = trades[trades['text'].str.contains('Funding')]
trades = trades[trades['execType'] == 'Trade']
trades = trades[trades['text'].str.contains('API')]
trades['exchange'] = 'BITMEX'
trades['execComm'] = trades['execComm'] * 0.00000001
trades['commissionAsset'] = 'XBT'
trades = trades.rename(columns={'execComm': 'commission', 'lastQty': 'qty'})
trades['side'] = trades['side'].str.upper()
trades = trades[['transactTime', 'exchange', 'symbol', 'side', 'price', 'qty', 'commission', 'commissionAsset']]

funding['exchange'] = 'BITMEX'
funding['execComm'] = funding['execComm'] * 0.00000001
funding['asset'] = 'XBT'
funding = funding.rename(columns={'lastQty': 'qty'})
funding = funding[['transactTime', 'exchange', 'symbol', 'qty', 'price', 'asset']]